<h1>Reasoning</h1>

The <b>Unified Rule Engine (URE)</b> is a generic opencog rule engine operating on the AtomSpace. Two chaining modes are currently supported, Forward Chaining and Backward Chaining.

In this tutorial, we are going to have a quick look at how we can do both forward and backward chaining, follow by a simple example using PLN.

In [2]:
(use-modules
  (opencog)
  (opencog query)
  (opencog rule-engine))

Now let's create the knowledge base, characterising frogs and spiders.

In [3]:
; Frog croaks
(Evaluation (stv 1.0 1.0)
   (Predicate "croaks") (Concept "frog"))

; Spider bites
(Evaluation (stv 1.0 1.0)
   (Predicate "bites") (Concept "spider"))

; Frog is green
(Inheritance (stv 1.0 1.0)
   (Concept "frog") (Concept "green"))

; Spider is brown
(Inheritance (stv 1.0 1.0)
   (Concept "spider") (Concept "brown"))

; Frog eats files
(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies") (Concept "frog"))

; Spider eats files
(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies") (Concept "spider"))

(EvaluationLink (stv 1 1)
   (PredicateNode "eats_flies")
   (ConceptNode "spider")
)


Next, let's create the inference rules. There are rules that are ready to use in URE, but to make it clearer for demostration purpose, let's simplified the process and create the our own rules for this tutorial:

In [4]:
(define if-croaks-and-eats-flies-then-frog-rule
  (BindLink
    (Variable "$X")
    (And
      (Evaluation
        (Predicate "croaks")
        (Variable "$X"))
      (Evaluation
        (Predicate "eats_flies")
        (Variable "$X")))
    (Inheritance
      (Variable "$X")
      (Concept "frog")))
)

(define if-croaks-and-eats-flies-then-frog-rule-name (DefinedSchema "if-croaks-and-eats-flies-then-frog-rule"))
(Define if-croaks-and-eats-flies-then-frog-rule-name if-croaks-and-eats-flies-then-frog-rule)

(define if-frog-then-green-rule
  (Bind
    (Variable "$X")
    (Inheritance
      (Variable "$X")
      (Concept "frog"))
    (Inheritance
      (Variable "$X")
      (Concept "green")))
)

(define if-frog-then-green-rule-name (DefinedSchema "if-frog-then-green-rule"))
(Define if-frog-then-green-rule-name if-frog-then-green-rule)

(DefineLink
   (DefinedSchemaNode "if-frog-then-green-rule")
   (BindLink
      (VariableNode "$X")
      (InheritanceLink
         (VariableNode "$X")
         (ConceptNode "frog")
      )
      (InheritanceLink
         (VariableNode "$X")
         (ConceptNode "green")
      )
   )
)


And then create our own rule base and add those rules to it:

In [5]:
(define frog-rb (Concept "frog-rb"))

(ure-add-rules frog-rb
  (list
    (cons if-croaks-and-eats-flies-then-frog-rule-name (stv 0.9 1))
    (cons if-frog-then-green-rule-name (stv 0.5 1))))

(ure-set-maximum-iterations frog-rb 20)

(ExecutionLink
   (SchemaNode "URE:maximum-iterations")
   (ConceptNode "frog-rb")
   (NumberNode "20.000000")
)


We also need to specifiy the source for forward chaining, to see what we can infer from it based on the knowledge we have.

In [6]:
(define source
  (Evaluation (stv 1.0 1.0)
     (Predicate "croaks")
     (Concept "frog")))

Finally do forward chaining using `cog-fc`:

In [7]:
(cog-fc frog-rb source)

(SetLink
   (InheritanceLink
      (ConceptNode "frog")
      (ConceptNode "frog")
   )
   (InheritanceLink (stv 1 1)
      (ConceptNode "frog")
      (ConceptNode "green")
   )
)


We can do backward chaining in a similar manner, but this time, instead of starting from a source, we start from the target -- what we want to find out.

So let's define the target, see what is green in color:

In [8]:
(define target (Inheritance (Variable "$what") (Concept "green")))

Additionally declare the variables involved:

In [9]:
(define var_decl (TypedVariable (Variable "$what") (Type "ConceptNode")))

And finally, do backward chaining using `cog-bc`:

In [10]:
(cog-bc frog-rb target #:vardecl var_decl)

(SetLink
   (InheritanceLink (stv 1 1)
      (ConceptNode "frog")
      (ConceptNode "green")
   )
)


<b>Probabilistic Logic Networks (PLN)</b> is a novel conceptual, mathematical and computational approach to uncertain inference that is able to encompass within uncertain logic such ideas as induction, abduction, analogy, fuzziness and speculation, and reasoning about time and causality.

Firstly, let's say we know that:

- Sacrates is a man
- Men are mortal

to some degree, we want to find out to what degree <b>Socrates is mortal</b>.

In [24]:
; Setup for PLN
(load-from-path "/home/relex/projects/opencog/opencog/pln/pln-config.scm")

; Rule base -- all the rules that inherit from it will be considered
(define rb (ConceptNode "PLN"))

; Knowledge with uncertainty
(Inheritance (stv 0.97 0.92)
  (Concept "Socrates")
  (ConceptNode "man"))

(Inheritance (stv 0.98 0.94)
  (Concept "man")
  (Concept "mortal"))

(InheritanceLink (stv 0.98 0.94)
   (ConceptNode "man")
   (ConceptNode "mortal")
)


Then set the target:

In [21]:
(define target
  (Inheritance
    (Concept "Socrates")
    (Concept "mortal")))

And finally call the PLN backward chainer:

In [25]:
(cog-bc rb target)

(SetLink
   (InheritanceLink (stv 0.9508 0.828)
      (ConceptNode "Socrates")
      (ConceptNode "mortal")
   )
)
